# Setting up access to Keyspace in Jupyter notebook

## Prerequisites
Two files are required for accessing Keyspace in Jupyter notebook:  

1. Starfield digital certificate
2. AWS credentials for Keyspace

More details about the setup can be found in [Using a Cassandra Python client](https://docs.aws.amazon.com/keyspaces/latest/devguide/using_python_driver.html#python_SigV4).

### Starfield digital certificate
The digital certificate can be downloaded by:

In [ ]:
!curl https://certs.secureserver.net/repository/sf-class2-root.crt -O

### AWS credentials
The AWS credentials file can be downloaded via Canvas.  Place this `credentials` file under `.aws/` folder on your local machine, or your EC2 instance, so that AWS knows where to look for the credentials.

**MAJOR NOTE**: **Do not** store this credentials anywhere that is publicly accessible, github, public S3 bucket, etc.  That is the primary reason why this file is only available on Canvas.

## Sample code for connection

1. Install `cassandra-sigv4` via the following command:

In [ ]:
%pip install cassandra-sigv4

2. Set up a `boto3` session and a Cassandra cluster (the Python way of interacting with AWS).

In [ ]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED
from cassandra_sigv4.auth import SigV4AuthProvider
import boto3

# ssl setup
ssl_context = SSLContext(PROTOCOL_TLSv1_2)
ssl_context.load_verify_locations('sf-class2-root.crt')  # change your file path for locating the certificate
ssl_context.verify_mode = CERT_REQUIRED

# boto3 session setup
boto_session = boto3.Session(region_name="us-east-2")  # this AWS credentials is specific to `us-east-2` region

In [ ]:
# authorization setup with SigV4
auth_provider = SigV4AuthProvider(boto_session)

In [ ]:
#cluster setup 
cluster = Cluster(['cassandra.us-east-2.amazonaws.com'], 
                  ssl_context=ssl_context, 
                  auth_provider=auth_provider, 
                  port=9142)  # TLS only communicates on port 9142

## Working with Cassandra (AWS Keyspace)

In [ ]:
# establishing connection to Keyspace
session = cluster.connect()

In [ ]:
# Insert any CQL queries between .connect() and .shutdown()

# For example, show all keyspaces created
r = session.execute('''
    SELECT * FROM system_schema.keyspaces;
    ''')
print(r.current_rows)

In [ ]:
# For example, create a keyspace for HW2
r = session.execute('''
    CREATE KEYSPACE IF NOT EXISTS de300_demo 
    WITH replication = {'class': 'SingleRegionStrategy'};
    ''')
print(r.current_rows)

## Shutdown your Cassandra connection

In [ ]:
session.shutdown()